## Point
What is symptom terminology in the input string?
Also, relationship similarity of words is important to seach information


### 1. 명확한 symptom으로 이루어진 문장으로 입력받는 경우
I have **cough, phlegm, sore throat, rash, and headaches.** Also, 

### 2. Symptom을 비유하듯이 표현한 경우
My **neck and armpit is swollen**, and **red**

또는

(감기 걸린 것 처럼, 몸이 으실으실하다)
My body is **cold shivers** like fever.

### 3. 단어(단답)로 입력받는 경우
Depression
Rapid heart rate
Annoyed
Anxiety



In [9]:
input1 = "I have cough, phlegm, sore throat, rash, blar, today i see the movie and headaches"
input2 = "My neck and armpit is swollen and red"
input3 = "Depression | Rapid heart rate | Annoyed | Anxiety"
inputEx = "SO MUCH PAIN! In the last 2 years I have suffered with a brain tumour so have been in a LOT of pain to the point of morphine everyday for a year. Then I had brain surgery...but the pain from this pill came pretty close!! In serious pain to the point of blacking out hot and cold shivers and just sat in pain feeling like trapped wind/indigestion it&#039;s just.. Ahhhh!!!! Don&#039;t take this!!!"

input1 = input1.lower()
input2 = input2.lower()
input3 = input3.lower()


## Tokenization, stemming, lemmatization, and stopword

원형으로 바꾼뒤에 어간 추출해야할듯

In [10]:
!pip show tensorflow

import re
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

#engLemm = WordNetLemmatizer()
engStopword = set(stopwords.words('english'))
pst = PorterStemmer()

def tokenize(input, engStopword)->list:
    tokenizeWord = text_to_word_sequence(input)
    #tokenizeWord = engLemm.lemmatize(tokenizeWord)
    result = []
    for word in tokenizeWord:
        if word not in engStopword:
            result.append(word)
    
    return result

input1 = tokenize(input1, engStopword)
input2 = tokenize(input2, engStopword)
input3 = tokenize(input3, engStopword)

stemmedInput1 = [pst.stem(token) for token in input1]
stemmedInput2 = [pst.stem(token) for token in input2]
stemmedInput3 = [pst.stem(token) for token in input3]
# nltk의 stem을 쓴 이유가 기억 나질 않아 (찾아야 함)
        
print(stemmedInput1)
print(stemmedInput2)
print(stemmedInput3)

Name: tensorflow
Version: 2.4.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/JunKi/opt/anaconda3/lib/python3.8/site-packages
Requires: gast, tensorflow-estimator, h5py, flatbuffers, numpy, astunparse, termcolor, grpcio, protobuf, typing-extensions, absl-py, google-pasta, wrapt, six, tensorboard, keras-preprocessing, wheel, opt-einsum
Required-by: 
['cough', 'phlegm', 'sore', 'throat', 'rash', 'blar', 'today', 'see', 'movi', 'headach']
['neck', 'armpit', 'swollen', 'red']
['depress', 'rapid', 'heart', 'rate', 'annoy', 'anxieti']


## Bi-gram으로 변환
sore throat와 같이 symptom이 2개의 corpus로 이루어질 수 있을것 같음
###### (?)빨갛게 부어오른 피부 는 세개지 않나?

In [11]:
biInput1 = list(zip(stemmedInput1, stemmedInput1[1:]))
biInput2 = list(zip(stemmedInput2, stemmedInput2[1:]))
biInput3 = list(zip(stemmedInput3, stemmedInput3[1:]))

def change(tupleData)->list:
    result = []
    for listData in tupleData:
        tmp = list(listData)
        result.append(" ".join(listData))
    return result
    
biInput1 = change(biInput1)
biInput2 = change(biInput2)
biInput3 = change(biInput3)

print(biInput1,"\n",biInput2,"\n",biInput3)

['cough phlegm', 'phlegm sore', 'sore throat', 'throat rash', 'rash blar', 'blar today', 'today see', 'see movi', 'movi headach'] 
 ['neck armpit', 'armpit swollen', 'swollen red'] 
 ['depress rapid', 'rapid heart', 'heart rate', 'rate annoy', 'annoy anxieti']


## Knowledge Base 긁어오기

In [19]:
#pip install --upgrade beautifulsoup4==4.4.1
#pip install --upgrade "html5lib==1.0b8"
!pip install html_table_parser
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser

url = "https://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html"

result = urlopen(url)
html = result.read()
soup = BeautifulSoup(html, 'html.parser')

temp = soup.find_all('table')
p = parser.make2d(temp[0])

df = pd.DataFrame(p[1:],columns = p[0])

df['Symptom'] = df['Symptom'].str.replace('\d+', '')
df['Symptom'] = df['Symptom'].str.replace('\s', ' ')
df['Symptom'] = df['Symptom'].str.replace('UMLS', '')
df['Symptom'] = df['Symptom'].str.replace(':C_', '')

print(df['Symptom'])

  Created wheel for html-table-parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4646 sha256=50dcecc06b9c147b768a392c46bc4f1f3d33473a962e23c6d0834fc96cfbd5bd
  Stored in directory: /Users/JunKi/Library/Caches/pip/wheels/b8/a6/f0/e77452c538f4ab23004d6d23be1a38e39cfd45ab8a80f6709d
Successfully built html-table-parser
0                pain   chest
1       shortness   of breath
2                   dizziness
3                    asthenia
4                        fall
                ...          
1861      bedridden^bedridden
1862               prostatism
1863          systolic murmur
1864                    frail
1865                    fever
Name: Symptom, Length: 1866, dtype: object


In [21]:
df

,Disease,Count of Disease\n Occurrence,Symptom
0,UMLS:C0020538_hypertensive\n disease,3363,pain chest
1,,,shortness of breath
2,,,dizziness
3,,,asthenia
4,,,fall
...,...,...,...
1861,,,bedridden^bedridden
1862,,,prostatism
1863,UMLS:C0011127_decubitus\n ulcer,42,systolic murmur
1864,,,frail


## 해당 단어가 knowledge base에 있는지 확인

In [20]:
def oneGramCheck(inputStr)->list:
    result = []
    for word in inputStr:
        for baseWord in df['Symptom']:
            if word in baseWord:
                result.append(word)
                break
    return result

result1 = oneGramCheck(stemmedInput1)
result2 = oneGramCheck(stemmedInput2)
result3 = oneGramCheck(stemmedInput3)
print(result1)
print(result2)
print(result3)



['cough', 'sore', 'throat', 'headach']
['neck', 'red']
['depress', 'rapid', 'heart', 'rate']


## Bi-gram에서도 확인

In [ ]:
biResult1 = oneGramCheck(biInput1)
print(biResult1)    

## Ah

1. one-gram에서는 swollen은 knowledge base에서 swelling, or swell로 표현됨. 따라서 원형비교나 다른 비교방법이 필요할듯
2. bi-gram의 경우에서도 corpust 구성 순서가 바뀌어 있을 수 있음

In [22]:
from difflib import SequenceMatcher

str1 = 'sore_throat'
str2 = 'throat sore'

ratio = SequenceMatcher(None, str1, str2).ratio()
print(ratio)

str1 = 'swell'
str2 = 'swollen'

ratio = SequenceMatcher(None, str1, str2).ratio()
print(ratio)

str1 = 'swollen'
str2 = 'swelling'

ratio = SequenceMatcher(None, str1, str2).ratio()
print(ratio)

0.5454545454545454
0.6666666666666666
0.6666666666666666


In [23]:
str1 = 'movie headache'
str2 = 'headache'
ratio = SequenceMatcher(None, str1, str2).ratio()
print(ratio)

0.7272727272727273


#### 0.5이상만 받으면 될듯

In [ ]:
def occureCheck(inputStr)->list:
    result = []
    for word in inputStr:
        print(word)
        for baseWord in df['Symptom']:
            ratio = SequenceMatcher(None, baseWord, word).ratio()
            if ratio > 0.66:
                result.append(word)
                break
    return result

result1 = occureCheck(biInput1)
result2 = occureCheck(biInput2)
result3 = occureCheck(biInput3)
print(result1)
print(result2)
print(result3)

## 근데 어차피 word2Vector등등 이용해서 review 데이터 쓴다면 개별 단어만 쓰면 되지 않나?
그래서 